In [1]:
#Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
#Read CSV into PANDAS
cars = '../data/vehicle-fuel-type-count-by-zip-code.csv'
geocodes = '../data/fips_zip_correct.csv'
cars_data = pd.read_csv(cars)
geocodes = pd.read_csv(geocodes)

In [3]:
#Get number of unique values for counties
geocodes1 = geocodes[geocodes['State'] == 6]
geocodes1.nunique()

zip       2448
State        1
County      58
dtype: int64

In [4]:
#Get rid of duplicate cars
cars_data.nunique()

Date             1
Zip Code      2915
Model Year      15
Fuel             9
Make           113
Duty             2
Vehicles      2375
dtype: int64

In [5]:
#Define number of cars and and length of car data
print('The number of rows in the car data is ' + str(len(cars_data)))
print('The number of cars in car data is ' + str(cars_data['Vehicles'].sum()))

The number of rows in the car data is 584338
The number of cars in car data is 30411713


In [6]:
#Getting the number of zips per county, by grouping by zip and then counting number of counties
geocodes = geocodes1.rename(columns = {'zip': 'Zip Code', 'County':'COUNTY'})
n_zip_county = geocodes.groupby('Zip Code').count().sort_values(['COUNTY'],ascending = False).reset_index()
n_zip_county.head()

,Zip Code,State,COUNTY
0,93252,4,4
1,95645,3,3
2,95329,3,3
3,95960,3,3
4,95690,3,3


In [7]:
#creating weights by doing 1/n
n_zip_county['Weights'] = 1/n_zip_county['COUNTY']
n_zip_county.head()

,Zip Code,State,COUNTY,Weights
0,93252,4,4,0.250000
1,95645,3,3,0.333333
2,95329,3,3,0.333333
3,95960,3,3,0.333333
4,95690,3,3,0.333333


In [8]:
#Renaming zip code for merge
cars_data =cars_data.rename(columns={'Zip Code': 'Zip Code'})
cars_data.head()
#Reducing zip codes to just zip code and weights
n_zip_county = n_zip_county[['Zip Code','Weights']].sort_values(['Zip Code'])
n_zip_county.head()

,Zip Code,Weights
1250,90001,1.0
2071,90002,1.0
1647,90003,1.0
1648,90004,1.0
1649,90005,1.0


In [9]:
#Mapping n_zips back into geocodes
geocodes_weights = pd.merge(geocodes,n_zip_county,
                            on = 'Zip Code',how = 'inner')
geocodes_weights.head()

,Zip Code,State,COUNTY,Weights
0,90016,6,37,1.0
1,90026,6,37,1.0
2,90018,6,37,1.0
3,96161,6,61,0.5
4,96161,6,57,0.5


In [ ]:
#merging weights/number of car regristrations by zip code into fips code
cars_merge_county = pd.merge(cars_data, geocodes_weights, how="inner", on=["Zip Code"])
cars_merge_county.sort_values(['Weights']).nunique()